# Genome Assembly Assignment

In this notebook, the functions required to complete **Assignment #6 (Python genome assembly)** are implemented.

**Author:** Elisabetta Roviera (s328422)  
**Course:** Bioquants
**Date:** June 9, 2025  
**Repository:** https://github.com/elisabettaroviera/Bioquants

## Problem 1: Read and Parse Input Data

Write a function `readDataFromFile(fileName)` that reads a file and returns a dictionary mapping read names to sequences.

In [1]:
def readDataFromFile(fileName):
    """
    Reads a file where each line has 'name:sequence' and returns a dict mapping names to sequences.
    """
    reads = {}
    with open(fileName, 'r') as f:
        for line in f:
            name, seq = line.strip().split(':')
            reads[name] = seq
    return reads

## Problem 2: Mean Read Length

Write a function `meanLength(fileName)` that returns the mean sequence length as a float.

In [2]:
def meanLength(fileName):
    """
    Calculates the mean length of reads in fileName.
    """
    reads = readDataFromFile(fileName)
    if not reads:
        return 0.0
    total_length = sum(len(seq) for seq in reads.values())
    return total_length / len(reads)

## Problem 3: Compute Overlap Between Two Reads

Write a function `getOverlap(left, right)` that returns the overlapping substring between the end of `left` and the start of `right`.

In [3]:
def getOverlap(left, right):
    """
    Returns the longest overlap between the 3' end of 'left' and the 5' start of 'right'.
    """
    max_overlap = ''
    max_len = min(len(left), len(right))
    for i in range(max_len, 0, -1):
        if left[-i:] == right[:i]:
            return left[-i:]
    return ''

## Problem 4: Compute All Pairwise Overlaps

Write a function `getAllOverlaps(reads)` that returns a nested dict of overlap lengths for each ordered read pair.

In [4]:
def getAllOverlaps(reads):
    """
    Given a dict of reads, returns a dict of dicts where d[a][b] is 
    the length of the overlap when a is left and b is right.
    """
    overlaps = {}
    for a, seq_a in reads.items():
        overlaps[a] = {}
        for b, seq_b in reads.items():
            if a == b:
                continue
            overlap_seq = getOverlap(seq_a, seq_b)
            overlaps[a][b] = len(overlap_seq)
    return overlaps

## Problem 5: Pretty-Print Overlap Matrix

Write a function `prettyPrint(overlaps)` that prints an aligned matrix of overlaps with dashes on the diagonal.

In [5]:
def prettyPrint(overlaps):
    """
    Prints a matrix of overlap lengths with '-' on the diagonal.
    """
    names = sorted(overlaps.keys(), key=lambda x: int(x))
    # Header
    print('   ' + ' '.join(f'{name:>3}' for name in names))
    for i in names:
        row = [f'{i:>3}']
        for j in names:
            if i == j:
                cell = ' - '
            else:
                cell = f'{overlaps[i].get(j, 0):>3}'
            row.append(cell)
        print(' '.join(row))

## Problem 6: Find the First Read

Write a function `findFirstRead(overlaps)` to identify the read with no significant overlaps (>2) as the first (leftmost).

In [6]:
def findFirstRead(overlaps):
    """
    Returns the name of the read that has no overlaps >2 when it is on the right side.
    """
    reads = overlaps.keys()
    for candidate in reads:
        # Check if no other read has overlap >2 to this candidate
        if all(overlaps[a].get(candidate, 0) <= 2 for a in reads if a != candidate):
            return candidate
    return None

## Problem 7: Determine Read Order

1. Write `findKeyForLargestValue(d)` to find the key with the max value in `d`.
2. Write `findOrder(name, overlaps)` that returns the ordered list of reads.

In [7]:
def findKeyForLargestValue(d):
    """
    Returns the key corresponding to the largest value in dict d.
    """
    return max(d, key=d.get)

def findOrder(name, overlaps):
    """
    Given the first read name and overlaps dict, returns the list of reads in order.
    """
    order = [name]
    current = name
    while True:
        next_overlaps = overlaps[current]
        # Filter only significant overlaps >2
        significant = {r: o for r, o in next_overlaps.items() if o > 2}
        if not significant:
            break
        next_read = findKeyForLargestValue(significant)
        order.append(next_read)
        current = next_read
    return order

## Problem 8: Assemble Genome

Write `assembleGenome(readOrder, reads, overlaps)` to reconstruct the full genome string.

In [8]:
def assembleGenome(readOrder, reads, overlaps):
    """
    Given the order of reads, the reads dict, and overlaps dict,
    returns the assembled genome sequence.
    """
    genome = reads[readOrder[0]]
    for prev, curr in zip(readOrder, readOrder[1:]):
        o_len = overlaps[prev][curr]
        genome += reads[curr][o_len:]
    return genome

## Example Usage

Load data, compute overlaps, determine order, and assemble genome.

In [9]:
# Example (uncomment and adjust filename if you have the file in the same directory)
filename = 'genome-assembly.txt'
reads = readDataFromFile(filename)
print('Mean length:', meanLength(filename))
overlaps = getAllOverlaps(reads)
prettyPrint(overlaps)
first_read = findFirstRead(overlaps)
order = findOrder(first_read, overlaps)
print('Read order:', order)
genome_sequence = assembleGenome(order, reads, overlaps)
print('Assembled genome:')
print(genome_sequence)

Mean length: 55.333333333333336
     1   2   3   4   5   6
  1  -    1   0   0   1  29
  2  13  -    1   0  21   0
  3   0   0  -    1   0   1
  4   1  17   1  -    2   0
  5  39   1   0   0  -   14
  6   0   0  43   1   0  - 
Read order: ['4', '2', '5', '1', '6', '3']
Assembled genome:
TGCGAGGGAAGTGAAGTATTTGACCCTTTACCCGGAAGAGCGGGACGCTGCCCTGCGCGATTCCAGGCTCCCCACGGGGTACCCATAACTTGACAGTAGATCTCGTCCAGACCCCTAGCTGGTACGTCTTCAGTAGAAAATTGTTTTTTTCTTCCAAGAGGTCGGAGTCGTGAACACATCAGT
